# NeMo voice swap demo
This notebook shows how to use NVIDIA NeMo (https://github.com/NVIDIA/NeMo) to construct a toy demo which will swap a voice in the audio fragment with a computer generated one.

At its core the demo does: 

* Automatic speech recognition of what is said in the file. E.g. converting audio to text
* Adding punctuation and capitalization to the text
* Generating spectrogram from resulting text
* Generating waveform audio from the spectrogram.

## Installation
NeMo can be installed via simple pip command.

In [1]:
BRANCH = 'r1.0.0rc1'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]


  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.0.0rc1) to /tmp/pip-install-vz230tv5/nemo-toolkit_943e2934aae340a296304f95e0fd3b20
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-vz230tv5/nemo-toolkit_943e2934aae340a296304f95e0fd3b20
  Running command git checkout -b r1.0.0rc1 --track origin/r1.0.0rc1
  Switched to a new branch 'r1.0.0rc1'
  Branch 'r1.0.0rc1' set up to track remote branch 'r1.0.0rc1' from 'origin'.
  Using cached numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached onnx-1.8.1-cp36-cp36m-manylinux2010_x86_64.whl (14.5 MB)
  Using cached pytorch_lightning-1.1.5-py3-none-any.whl (685 kB)
  Using cached torch-1.7.1-cp36-cp36m-manylinux1_x86_64.whl (776.8 MB)
  Using cached wget-3.2.zip (10 kB)
  Using cached wrapt-1.12.1.tar.gz (27 kB)
  Using cached ruamel.yaml-0.16.12-py2.py3-none-any.whl (111 kB)
  Using cached scikit_learn-0.24.1-cp36-cp36m-manylinux2010_x86_64.whl (22.2 MB)
  Using cached omegaconf-

     |████████████████████████████████| 4.1 MB 6.8 MB/s eta 0:00:01
  Using cached absl_py-0.11.0-py3-none-any.whl (127 kB)
  Using cached google_auth-1.27.0-py2.py3-none-any.whl (135 kB)
  Using cached tensorboard_plugin_wit-1.8.0-py3-none-any.whl (781 kB)
  Using cached Werkzeug-1.0.1-py2.py3-none-any.whl (298 kB)
  Using cached Markdown-3.3.4-py3-none-any.whl (97 kB)
  Using cached wheel-0.36.2-py2.py3-none-any.whl (35 kB)
  Using cached google_auth_oauthlib-0.4.2-py2.py3-none-any.whl (18 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Using cached cachetools-4.2.1-py3-none-any.whl (12 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached urllib3-1.26.3-py2.py3-none-any.whl (137 kB)
  Using cached idna

  Using cached shortuuid-1.0.1-py3-none-any.whl (7.5 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached subprocess32-3.5.4-py3-none-any.whl
     |████████████████████████████████| 159 kB 11.9 MB/s eta 0:00:01
  Using cached psutil-5.8.0-cp36-cp36m-manylinux2010_x86_64.whl (291 kB)
  Using cached promise-2.3-py3-none-any.whl
  Using cached pathtools-0.1.2-py3-none-any.whl
  Using cached sentry_sdk-0.20.3-py2.py3-none-any.whl (131 kB)
  Using cached gitdb-4.0.5-py3-none-any.whl (63 kB)
  Using cached smmap-3.0.5-py2.py3-none-any.whl (25 kB)
Using legacy 'setup.py install' for nemo-toolkit, since package 'wheel' is not installed.
Using legacy 'setup.py install' for antlr4-python3-runtime, since package 'wheel' is not installed.
Using legacy 'setup.py install' for future, since package 'wheel' is not installed.
Using legacy 'setup.py install' for sacremoses, since package 'wheel' is not installed.
Using legacy 'setup.py install' for idna-ssl, since package

In [2]:
# Ignore pre-production warnings
import warnings
warnings.filterwarnings('ignore')
import nemo
# Import Speech Recognition collection
import nemo.collections.asr as nemo_asr
# Import Natural Language Processing colleciton
import nemo.collections.nlp as nemo_nlp
# Import Speech Synthesis collection
import nemo.collections.tts as nemo_tts
# We'll use this to listen to audio
import IPython

[NeMo W 2021-02-28 14:44:10 experimental:28] Module <class 'nemo.collections.asr.models.clustering_diarizer.ClusteringDiarizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-02-28 14:44:11 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [3]:
# Download audio sample which we'll try
# This is a sample from LibriSpeech Dev Clean dataset - the model hasn't seen it before
Audio_sample = '2086-149220-0033.wav'
!wget https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
# Listen to it
IPython.display.Audio(Audio_sample)

--2021-02-28 14:44:51--  https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
Resolving dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)... 52.219.104.160
Connecting to dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)|52.219.104.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 237964 (232K) [audio/wav]
Saving to: ‘2086-149220-0033.wav’

2086-149220-0033.wa 100%[===================>] 232.39K   795KB/s    in 0.3s    

2021-02-28 14:44:52 (795 KB/s) - ‘2086-149220-0033.wav’ saved [237964/237964]



## Instantiate pre-trained NeMo models which we'll use
``from_pretrained(...)`` API downloads and initialized model directly from the cloud.

We will load audio_sample and convert it to text with QuartzNet ASR model (an action called transcribe).
To convert text back to audio, we actually need to generate spectrogram with Tacotron2 first and then convert it to actual audio signal using WaveGlow vocoder.

In [4]:
# Speech Recognition model - QuartzNet
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En").cuda()
# Punctuation and capitalization model
punctuation = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained(model_name='Punctuation_Capitalization_with_DistilBERT').cuda()
# Spectrogram generator which takes text as an input and produces spectrogram
spectrogram_generator = nemo_tts.models.Tacotron2Model.from_pretrained(model_name="Tacotron2-22050Hz").cuda()
# Vocoder model which takes spectrogram and produces actual audio
vocoder = nemo_tts.models.WaveGlowModel.from_pretrained(model_name="WaveGlow-22050Hz").cuda()

[NeMo I 2021-02-28 14:45:13 cloud:56] Found existing object /home/sourav/.cache/torch/NeMo/NeMo_1.0.0rc1/QuartzNet15x5Base-En/00869f9c89b8393ca3de640e0c536bd2/QuartzNet15x5Base-En.nemo.
[NeMo I 2021-02-28 14:45:13 cloud:62] Re-using file from: /home/sourav/.cache/torch/NeMo/NeMo_1.0.0rc1/QuartzNet15x5Base-En/00869f9c89b8393ca3de640e0c536bd2/QuartzNet15x5Base-En.nemo
[NeMo I 2021-02-28 14:45:13 common:615] Instantiating model from pre-trained checkpoint
[NeMo I 2021-02-28 14:45:14 features:239] PADDING: 16
[NeMo I 2021-02-28 14:45:14 features:255] STFT using torch
[NeMo I 2021-02-28 14:45:17 modelPT:413] Model EncDecCTCModel was successfully restored from /home/sourav/.cache/torch/NeMo/NeMo_1.0.0rc1/QuartzNet15x5Base-En/00869f9c89b8393ca3de640e0c536bd2/QuartzNet15x5Base-En.nemo.
[NeMo I 2021-02-28 14:45:17 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemonlpmodels/versions/1.0.0a5/files/Punctuation_Capitalization_with_DistilBERT.nemo to /home/sourav/.cache/tor

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.
[NeMo W 2021-02-28 14:45:49 modelPT:146] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    text_file: text_train.txt
    labels_file: labels_train.txt
    shuffle: true
    num_samples: -1
    batch_size: 64
    
[NeMo W 2021-02-28 14:45:49 modelPT:153] Please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    ds_item: null
    text_file: text_dev.txt
    labels_file: labels_dev.txt
    shuffle: false
    num_samples: -1
    batch_size: 64
    
[NeMo W 2021-02-28 14:45:49 modelPT:1224] World size can only be set by PyTorch Lightning Trainer.


Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

[NeMo I 2021-02-28 14:46:15 modelPT:413] Model PunctuationCapitalizationModel was successfully restored from /home/sourav/.cache/torch/NeMo/NeMo_1.0.0rc1/Punctuation_Capitalization_with_DistilBERT/6906a3705a7989aa9995aed680fb5d22/Punctuation_Capitalization_with_DistilBERT.nemo.
[NeMo I 2021-02-28 14:46:15 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemottsmodels/versions/1.0.0a5/files/Tacotron2-22050Hz.nemo to /home/sourav/.cache/torch/NeMo/NeMo_1.0.0rc1/Tacotron2-22050Hz/59f6603364e11524c03c95920e7afaa0/Tacotron2-22050Hz.nemo
[NeMo I 2021-02-28 14:46:34 common:615] Instantiating model from pre-trained checkpoint
[NeMo I 2021-02-28 14:46:35 features:239] PADDING: 16
[NeMo I 2021-02-28 14:46:35 features:248] STFT using conv
[NeMo I 2021-02-28 14:46:36 modelPT:413] Model Tacotron2Model was successfully restored from /home/sourav/.cache/torch/NeMo/NeMo_1.0.0rc1/Tacotron2-22050Hz/59f6603364e11524c03c95920e7afaa0/Tacotron2-22050Hz.nemo.
[NeMo I 2021-02-28 14:46:3

## Using the models

In [5]:
# Convert our audio sample to text
files = [Audio_sample]
raw_text = ''
text = ''
for fname, transcription in zip(files, quartznet.transcribe(paths2audio_files=files)):
  raw_text = transcription

# Add capitalization and punctuation
res = punctuation.add_punctuation_capitalization(queries=[raw_text])
text = res[0]
print(f'\nRaw recognized text: {raw_text}. \nText with capitalization and punctuation: {text}')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2021-02-28 14:52:19 patch_utils:50] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.


[NeMo I 2021-02-28 14:52:19 punctuation_capitalization_model:385] Using batch size 1 for inference
[NeMo I 2021-02-28 14:52:19 punctuation_capitalization_dataset:134] Max length: 27
[NeMo I 2021-02-28 14:52:19 data_preprocessing:297] Some stats of the lengths of the sequences:
[NeMo I 2021-02-28 14:52:19 data_preprocessing:303] Min: 27 |                  Max: 27 |                  Mean: 27.0 |                  Median: 27.0
[NeMo I 2021-02-28 14:52:19 data_preprocessing:305] 75 percentile: 27.00
[NeMo I 2021-02-28 14:52:19 data_preprocessing:306] 99 percentile: 27.00
[NeMo I 2021-02-28 14:52:19 punctuation_capitalization_dataset:165] 0 are longer than 27
[NeMo I 2021-02-28 14:52:19 punctuation_capitalization_dataset:168] *** Example ***
[NeMo I 2021-02-28 14:52:19 punctuation_capitalization_dataset:169] i: 0
[NeMo I 2021-02-28 14:52:19 punctuation_capitalization_dataset:170] subtokens: [CLS] well i don ' t wish to see it any more observed phoebe turning away her eyes it is certainly ver

In [6]:
# A helper function which combines Tacotron2 and WaveGlow to go directly from 
# text to audio
def text_to_audio(text):
  parsed = spectrogram_generator.parse(text)
  spectrogram = spectrogram_generator.generate_spectrogram(tokens=parsed)
  audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)
  return audio.to('cpu').numpy()

## Results

In [7]:
# This is our original audio sample
IPython.display.Audio(Audio_sample)

In [8]:
# This is what was recognized by the ASR model
print(raw_text)

well i don't wish to see it any more observed phoebe turning away her eyes it is certainly very like the old portrait


In [9]:
# This is how punctuation model changed it
print(text)

Well, I don't wish to see it any more, observed Phoebe, turning away her eyes. It is certainly very like the old portrait.


Compare how the synthesized audio sounds when using text with and without punctuation.

In [10]:
# Without punctuation
IPython.display.Audio(text_to_audio(raw_text), rate=22050)

In [11]:
# Final result - with punctuation
IPython.display.Audio(text_to_audio(text), rate=22050)

## Next steps
A demo like this is great for prototyping and experimentation. However, for real production deployment, you would want to use a service like [NVIDIA Jarvis](https://developer.nvidia.com/nvidia-jarvis).

**NeMo is built for training.** You can fine-tune, or train from scratch on your data all models used in this example. We recommend you checkout the following, more in-depth, tutorials next:

* [NeMo fundamentals](https://colab.research.google.com/github/NVIDIA/NeMo/blob/main/tutorials/00_NeMo_Primer.ipynb)
* [NeMo models](https://colab.research.google.com/github/NVIDIA/NeMo/blob/main/tutorials/01_NeMo_Models.ipynb)
* [Speech Recognition](https://colab.research.google.com/github/NVIDIA/NeMo/blob/main/tutorials/asr/01_ASR_with_NeMo.ipynb)
* [Punctuation and Capitalization](https://colab.research.google.com/github/NVIDIA/NeMo/blob/main/tutorials/nlp/Punctuation_and_Capitalization.ipynb)
* [Speech Synthesis](https://colab.research.google.com/github/NVIDIA/NeMo/blob/main/tutorials/tts/1_TTS_inference.ipynb)


You can find scripts for training and fine-tuning ASR, NLP and TTS models [here](https://github.com/NVIDIA/NeMo/tree/main/examples). 

That's it folks! Head over to NeMo GitHub for more examples: https://github.com/NVIDIA/NeMo